In [0]:
# Importing the necessary libraries
import os
import numpy as np
from sklearn.cluster import KMeans
from skimage.io import imread, imsave
from skimage.util import img_as_float
from scipy.signal import wiener
from scipy.ndimage import distance_transform_edt
from skimage.measure import label, regionprops
from skimage.morphology import disk, white_tophat, black_tophat, reconstruction

In [ ]:
# Defining the directory where the MRI images are located
mri_images_dir = "/Inputs/MRI Images/"
mri_file_list = os.listdir(mri_images_dir)

# Defining the output directories
sat_output_path = "/Outputs K-MEANS/SAT Masks/"
vat_output_path = "/Outputs K-MEANS/VAT Masks/"
total_output_path = "/Outputs K-MEANS/Total Masks/"

# Ensuring that the directories exist, if not, we create them
os.makedirs(sat_output_path, exist_ok=True)
os.makedirs(vat_output_path, exist_ok=True)
os.makedirs(total_output_path, exist_ok=True)

# Loop through each MRI image
for mri_file in mri_file_list:
    # Forming a complete filepath for each MRI image
    file_path = os.path.join(mri_images_dir, mri_file)

    # Load the MRI image
    img = img_as_float(imread(file_path, as_gray=True))

    # Apply Wiener filter for noise reduction
    window_size = (6, 6)
    img_filtered = wiener(img, window_size)

    # Apply morphological operations: Top-Hat and Bottom-Hat
    selem = disk(50)
    top_hat = white_tophat(img_filtered, selem)
    bot_hat = black_tophat(img_filtered, selem)

    # Enhance the image using the results from the Top-Hat and Bottom-Hat operations
    enhanced_img = img + top_hat - bot_hat
    enhanced_img = np.clip(enhanced_img, 0, 1)

    # Perform contrast enhancement
    contrast_img = enhanced_img - bot_hat
    contrast_img = np.clip(contrast_img, 0, 1)

    # Reshape the image for clustering
    img_flat = contrast_img.ravel()
    img_flat = img_flat.reshape(-1, 1)

    # Number of clusters for K-Means
    n_clusters = 2

    # Apply K-Means to the reshaped image
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(img_flat)

    # Get the cluster labels
    labels_flat = kmeans.labels_
    labels = labels_flat.reshape(img.shape)

    # Make sure the background is always labelled as 0
    if labels[10, 10] == 1:
        labels = 1 - labels

    # Define a threshold to identify the arms
    threshold = 0.1
    img_bw = img > threshold

    # Remove arms from the image
    seed = np.copy(img_bw)
    seed[1:-1, 1:-1] = img_bw.max()
    img_bw = reconstruction(seed, img_bw, method='erosion')

    # Identify and label different regions in the image
    labeled_image = label(img_bw)
    areas = regionprops(labeled_image)

    # Find the region with the largest area
    max_area = max(area['area'] for area in areas)
    pos = [area.label for area in areas if area['area'] == max_area]

    # Create a binary image with only the torso
    img_obj = np.isin(labeled_image, pos)
    img_without_arms = img_obj

    # Remove the arms by multiplying the torso image with the original image (similar to logical AND)
    labels = labels * img_without_arms

    # Identify and label different regions in the image again
    labeled_img, _ = label(labels, connectivity=2, return_num=True)
    props = regionprops(labeled_img)

    # Create a copy of the label image
    img_area1 = labels.copy()

    # Calculate areas and centers of the different regions
    areas = [prop.area for prop in props]
    centroids = [prop.centroid for prop in props]

    # Define the image center
    image_center = np.array(labels.shape) / 2

    # Calculate the distances from each region's centroid to the image center
    distances = [np.linalg.norm(np.array(centroid) - image_center) for centroid in centroids]

    # Identify the two largest regions
    sorted_area_indices = np.argsort(areas)[::-1]

    # Determine which of the two largest regions is closer to the image center
    distance_first = distances[sorted_area_indices[0]]
    distance_second = distances[sorted_area_indices[1]]

    # Assign the labels for SAT and VAT based on their relative positions
    if distance_first < distance_second:
        sat_label = sorted_area_indices[0] + 1
        vat_label = sorted_area_indices[1] + 1
    else:
        sat_label = sorted_area_indices[1] + 1
        vat_label = sorted_area_indices[0] + 1

    # Create binary images of the SAT and VAT
    img_area1[labeled_img != sat_label] = 0
    vat_image = np.copy(labels)
    vat_image[img_without_arms == 0] = 0

    # Invert the SAT image
    img_cmma_inverted = 1 - img_area1
    vat_image[img_cmma_inverted == 0] = 0

    # Perform distance transform
    dist_transform = distance_transform_edt(img_without_arms)
    img_without_arms = dist_transform > 5

    # Apply the binary torso image to the VAT image
    vat_image = vat_image * img_without_arms
    sat_image = img_area1

    # Save the SAT, VAT and total masks
    imsave(os.path.join(sat_output_path, mri_file), (sat_image * 255).astype(np.uint8))
    imsave(os.path.join(vat_output_path, mri_file), (vat_image * 255).astype(np.uint8))
    imsave(os.path.join(total_output_path, mri_file), (labels * 255).astype(np.uint8))